# COCO to YOLO labels converter

## Import COCO labels from filtered JSON FILE

### (MAIN) Filtering was done with filter.py file. Terminal command: *python filter.py --input_json c:\users\you\annotations\instances_train2017.json --output_json c:\users\you\annotations\filtered.json --categories person dog cat* GET FITERED JSON THEN JSON2YOLO

In [ ]:
!pip install pylabel

In [ ]:
from pylabel import importer
dataset = importer.ImportCoco(path='instances_val2014.json', path_to_images='smp')

In [ ]:
dataset.df.head(5)

## Reindexing dataset (to start from 0, delete gaps)

In [ ]:
df = dataset.df
cat_id_index = 0
"""
Reindex the values of the cat_id column so that that they start from an int (usually 0 or 1) and
then increment the cat_ids to index + number of categories.
It's useful if the cat_ids are not continuous, especially for dataset subsets,
or combined multiple datasets. Some models like Yolo require starting from 0 and others
like Detectron require starting from 1.
"""
assert isinstance(cat_id_index, int), "cat_id_index must be an int."

# Convert empty strings to NaN and drop rows with NaN cat_id
df_copy = df.replace(r"^\s*$", np.nan, regex=True)
df_copy = df_copy[df.cat_id.notnull()]
# Coerce cat_id to int
df_copy["cat_id"] = pd.to_numeric(df_copy["cat_id"])
# Map cat_ids to the range [cat_id_index, cat_id_index + num_cats)
unique_ids = np.sort(df_copy["cat_id"].unique())
ids_dict = dict((v, k)
                for k, v in enumerate(unique_ids, start=cat_id_index))
df_copy["cat_id"] = df_copy["cat_id"].map(ids_dict)

# Write back to the original dataframe
df["cat_id"] = df_copy["cat_id"]

## Export as YOLO format txt files

In [ ]:
dataset.path_to_annotations = "annotations/yolo"
#############################################################
#dataset.export.ExportToYoloV5()[0] #cat_id_index (int) if needs to start from 0 or 1 training/dataset.yaml
dataset.export.ExportToYoloV5(cat_id_index=0)[0]

In [ ]:
!zip -r /content/annotations_val.zip /content/training/labels